In [147]:
# my horrible spaghetti that imports and prepares the data, and then splits it into training, validation and test sets 

import importlib
import os
import fnmatch
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import csv
from sklearn import tree
import sys
sys.stdout.flush()
import math
import matplotlib.pyplot as plt
from matplotlib.table import Table
import segment_cluster as sc
import importlib
importlib.reload(sc)
from sklearn.cluster import KMeans
import pandas as pd
np.random.seed(0)

cwd = os.getcwd()
if cwd.split("/")[1] == "home":
    data_path="/home/jkok1g14/Documents/GRS1915+105/data/Std1_PCU2"
elif cwd.split("/")[1] == "export":
    data_path="/export/data/jakubok/GRS1915+105/Std1_PCU2"
else:
    print("Set the path for data directory!", Flush=True)

clean_belloni = open('1915Belloniclass_updated.dat')
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
    if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state

available = []
pool=[]

#/home/jkok1g14/Documents/GRS1915+105/data
#/export/data/jakubok/GRS1915+105/Std1_PCU2
for root, dirnames, filenames in os.walk(data_path):
    for filename in fnmatch.filter(filenames, "*_std1_lc.txt"):
        available.append(filename)
for ob, state in ob_state.items():
    if ob+"_std1_lc.txt" in available:
        pool.append(ob)  

#create a list of arrays with time and counts for the set of Belloni classified observations
lc_dirs=[]
lcs=[]
ids=[]
for root, dirnames, filenames in os.walk(data_path):    
    for filename in fnmatch.filter(filenames, "*_std1_lc.txt"):
        if filename.split("_")[0] in pool:
            lc_dirs.append(os.path.join(root, filename))

            
#make 2D arrays for light curves, with columns of counts and time values
for lc in lc_dirs:
    ids.append(lc.split("/")[-1].split("_")[0])
    f=np.loadtxt(lc)
    f=np.transpose(f)#,axis=1)
    f=f[0:2]
    ###1s average and time check to eliminate points outside of GTIs
    f8t = np.mean(f[0][:(len(f[0])//8)*8].reshape(-1, 8), axis=1)
    f8c = np.mean(f[1][:(len(f[1])//8)*8].reshape(-1, 8), axis=1)
    #f8c=f8c-np.mean(f8c)#normalisation/mean centering/whatever you desire most
    rm_points = []
    skip=False
    for i in range(len(f8t)-1):
        if skip==True:
            skip=False
            continue
        delta = f8t[i+1]-f8t[i]
        if delta > 1.0:
            rm_points.append(i+1)
            skip=True
            
####### normalise the count rates! think about the effect of 0-1 normalisation on the distance calculation
####### due to the energy integration in Std1 diefferences between different epochs shouldn't matter; there would be very few photons found at the extremes of the range            
    times=np.delete(f8t,rm_points)
    counts=np.delete(f8c,rm_points)
    lcs.append(np.stack((times,counts)))
#a list of light curve 2D arrays

lc_classes=[]
for i in ids:
    lc_classes.append(ob_state[i])

drop_classes=[]
for clas, no in Counter(lc_classes).items():
    if no<7:
        drop_classes.append(clas)

lcs_abu = []
classes_abu = []
ids_abu = []
for n, lc in enumerate(lc_classes):
    if lc not in drop_classes:
        classes_abu.append(lc)
        lcs_abu.append(lcs[n])
        ids_abu.append(ids[n])  
#a list of light curve 2D arrays of classes with at least 7 light curves


In [148]:
lcs_abu_std=sc.scaling(lcs_abu, method="standard")
# data is standardised, x_i_stand = (x_i - x_mean)/x_std
# mean+n*sigma is going to be the assumed maximum count rate that will be used to normalise the data

x_train, x_test, y_train, y_test, id_train, id_test = train_test_split(lcs_abu_std, classes_abu, ids_abu, test_size=0.25, stratify=classes_abu)
#x_train, x_test, y_train, y_test, id_train, id_test = train_test_split(lcs_abu, classes_abu, ids_abu, test_size=0.25, random_state=0, stratify=classes_abu)


In [149]:
importlib.reload(sc)
k_clusters=[5, 25, 50]
seg_lens=[8, 60, 100]
seg_len=1
seg_slides=[1, int(seg_len*0.25), int(seg_len*0.5)]
model_class="rho"
for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        # create a wave of the required length to center the segments
        
        for slide_id, seg_slide in enumerate(seg_slides):
            #bootstrapping
            for CV_id in range(10):
                x_cvtrain, x_valid, y_cvtrain, y_valid, id_cvtrain, id_valid = train_test_split(x_train, y_train, id_train, test_size=0.33, stratify=y_train)
                ##train the model
                #loop throught the light curves of a given class and segments them
                training_ids=np.where(np.array(y_cvtrain)=='{}'.format(model_class))[0]
                all_train_segments=[]
                for ts_i in training_ids:
                    ts=x_cvtrain[ts_i]
                    train_segments=sc.segmentation(ts, seg_len, seg_slide, time_stamps=True)
                    all_train_segments.append(train_segments)
                all_train_segments=np.vstack(all_train_segments)
                #cluster the segments
                cluster=KMeans(n_clusters=k_cluster, random_state=0)
                cluster.fit(all_train_segments[:,1,:])
                
                ### reconstruction
                classes=list(set(y_valid))
                
                #loop through light curves of every class
                for n_test, test_class in enumerate(classes):
                    testing_ids=np.where(np.array(y_valid)=='{}'.format(test_class))[0]
                    reco_error=[]
                    for ts_id in testing_ids:
                        test_ts=x_valid[ts_id]
                        test_segments= sc.segmentation(test_ts, seg_len, int(seg_len/2) , time_stamps=True)
                        reco = sc.reconstruct(test_segments, test_ts, cluster, rel_offset=False)
                        error=np.sqrt(np.mean((test_ts[1][seg_len:-seg_len]-reco[1][seg_len:-seg_len])**2))
                        reco_error.append((k_id,len_id,slide_id,CV_id,ts_id, error))
                        print((k_cluster,seg_len,seg_slide,CV_id,ts_id, error))
results=np.array(reco_error)
np.savetxt("valid_results_20190402.csv", results, delimiter=",") 

(0, 0, 0, 0, 7, 0.2813880194783041)
(0, 0, 0, 0, 19, 0.3423159757458115)
(0, 0, 0, 0, 23, 0.28210720316651117)
(0, 0, 0, 0, 44, 0.28299650582287544)
(0, 0, 0, 0, 49, 0.2928905573080922)
(0, 0, 0, 0, 2, 0.1332354930010893)
(0, 0, 0, 0, 3, 0.46170974622094063)
(0, 0, 0, 0, 6, 0.13427408216799028)
(0, 0, 0, 0, 8, 0.34107423460641123)
(0, 0, 0, 0, 9, 0.4788408636263153)
(0, 0, 0, 0, 11, 0.09318039188896912)
(0, 0, 0, 0, 12, 0.09234540171015018)
(0, 0, 0, 0, 14, 0.3561697633740944)
(0, 0, 0, 0, 16, 0.1404296283815975)
(0, 0, 0, 0, 21, 0.30653737876034165)
(0, 0, 0, 0, 24, 0.15976258923202924)
(0, 0, 0, 0, 28, 0.10736290429068866)
(0, 0, 0, 0, 31, 0.8591756333108702)
(0, 0, 0, 0, 33, 0.4413482552399607)
(0, 0, 0, 0, 37, 0.20625467715652415)
(0, 0, 0, 0, 38, 0.14136618316696578)
(0, 0, 0, 0, 40, 0.42704551325210005)
(0, 0, 0, 0, 41, 0.11511577522454891)
(0, 0, 0, 0, 48, 0.33607630174435676)
(0, 0, 0, 0, 0, 0.4304117034971438)
(0, 0, 0, 0, 1, 0.3134511965468264)
(0, 0, 0, 0, 4, 0.7580844636818

TypeError: 'float' object cannot be interpreted as an integer

In [156]:
x_train, x_test, y_train, y_test, id_train, id_test = train_test_split(lcs_abu_std, classes_abu, ids_abu, test_size=0.25, stratify=classes_abu)
#x_train, x_test, y_train, y_test, id_train, id_test = train_test_split(lcs_abu, classes_abu, ids_abu, test_size=0.25, random_state=0, stratify=classes_abu)

reco_error=[]
importlib.reload(sc)
k_clusters=[5, 25, 50]
seg_lens=[8, 60, 100]
model_class="rho"
for k_id, k_cluster in enumerate(k_clusters):
    for len_id, seg_len in enumerate(seg_lens):
        # calculate the slide values
        seg_slides=[1, int(seg_len*0.25), int(seg_len*0.5)]
        for slide_id, seg_slide in enumerate(seg_slides):
            #bootstrapping
            for CV_id in range(10):
                x_cvtrain, x_valid, y_cvtrain, y_valid, id_cvtrain, id_valid = train_test_split(x_train, y_train, id_train, test_size=0.33, stratify=y_train)
                ##train the model
                #loop throught the light curves of a given class and segments them
                training_ids=np.where(np.array(y_cvtrain)=='{}'.format(model_class))[0]
                all_train_segments=[]
                for ts_i in training_ids:
                    ts=x_cvtrain[ts_i]
                    train_segments=sc.segmentation(ts, seg_len, seg_slide, time_stamps=True)
                    all_train_segments.append(train_segments)
                all_train_segments=np.vstack(all_train_segments)
                #cluster the segments
                cluster=KMeans(n_clusters=k_cluster, random_state=0)
                cluster.fit(all_train_segments[:,1,:])
                
                ### reconstruction
                classes=list(set(y_valid))
                
                #loop through light curves of every class
                for n_test, test_class in enumerate(classes):
                    testing_ids=np.where(np.array(y_valid)=='{}'.format(test_class))[0]
                    for ts_id in testing_ids:
                        test_ts=x_valid[ts_id]
                        test_segments= sc.segmentation(test_ts, seg_len, int(seg_len/2) , time_stamps=True)
                        reco = sc.reconstruct(test_segments, test_ts, cluster, rel_offset=False)
                        error=np.sqrt(np.mean((test_ts[1][seg_len:-seg_len]-reco[1][seg_len:-seg_len])**2))
                        reco_error.append((k_id,len_id,slide_id,CV_id,ts_id, error))
                        print((k_cluster,seg_len,seg_slide,CV_id,ts_id, error))
results=np.array(reco_error)
#np.savetxt("valid_results_20190402.csv", results, delimiter=",") 

(5, 8, 1, 0, 0, 0.2776426955039365)
(5, 8, 1, 0, 9, 0.31909198970578756)
(5, 8, 1, 0, 14, 0.3163083047025516)
(5, 8, 1, 0, 20, 0.3674544619759929)
(5, 8, 1, 0, 44, 0.24955824970261276)
(5, 8, 1, 0, 2, 0.34395691260480954)
(5, 8, 1, 0, 8, 0.5884145498388741)
(5, 8, 1, 0, 10, 0.3619289939526382)
(5, 8, 1, 0, 11, 0.19270094443734628)
(5, 8, 1, 0, 15, 0.2344880595717194)
(5, 8, 1, 0, 16, 0.1043191530004725)
(5, 8, 1, 0, 18, 0.2884457581458273)
(5, 8, 1, 0, 19, 0.09911746741726206)
(5, 8, 1, 0, 22, 0.6137164497739014)
(5, 8, 1, 0, 24, 0.6218108741275961)
(5, 8, 1, 0, 25, 0.0947653650954596)
(5, 8, 1, 0, 26, 0.23577436580672376)
(5, 8, 1, 0, 29, 0.4922196954054451)
(5, 8, 1, 0, 34, 0.2218391843463207)
(5, 8, 1, 0, 39, 0.10625290608190102)
(5, 8, 1, 0, 40, 0.36232535704444035)
(5, 8, 1, 0, 46, 0.34802304189241257)
(5, 8, 1, 0, 47, 0.18067020192532016)
(5, 8, 1, 0, 48, 0.3293180057061221)
(5, 8, 1, 0, 5, 0.5013043702830962)
(5, 8, 1, 0, 21, 1.0335230443129446)
(5, 8, 1, 0, 28, 0.75920688935504

In [177]:
output=open("out_20190402.txt", "r")
output=np.asarray(output.read())
output

array('(5, 8, 1, 0, 21, 0.30871205638036653)\n(5, 8, 1, 0, 22, 0.38958703681113827)\n(5, 8, 1, 0, 35, 0.5177431671216929)\n(5, 8, 1, 0, 36, 0.7517435471544575)\n(5, 8, 1, 0, 44, 0.6048099010908803)\n(5, 8, 1, 0, 3, 0.3819914469250196)\n(5, 8, 1, 0, 4, 0.3591889294543705)\n(5, 8, 1, 0, 9, 0.7021716125075975)\n(5, 8, 1, 0, 20, 0.5551432820290458)\n(5, 8, 1, 0, 39, 0.2459297544375788)\n(5, 8, 1, 0, 15, 0.3697249355979346)\n(5, 8, 1, 0, 19, 0.3714456635206323)\n(5, 8, 1, 0, 27, 0.3291180171829786)\n(5, 8, 1, 0, 24, 0.6310033019436891)\n(5, 8, 1, 0, 49, 2.073930750799398)\n(5, 8, 1, 0, 31, 0.18383799840380913)\n(5, 8, 1, 0, 34, 0.17952417338542487)\n(5, 8, 1, 0, 16, 0.18984883959540624)\n(5, 8, 1, 0, 23, 0.370115090748285)\n(5, 8, 1, 0, 30, 0.3641741745569272)\n(5, 8, 1, 0, 33, 0.2824577803683068)\n(5, 8, 1, 0, 37, 0.3429927188745314)\n(5, 8, 1, 0, 5, 0.4197236203058596)\n(5, 8, 1, 0, 17, 0.3258777486275771)\n(5, 8, 1, 0, 25, 0.4393824728999776)\n(5, 8, 1, 0, 0, 0.37553061032920904)\n(5, 8,

''